In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

In [2]:
#Load the dataset
frame_data = np.load("../sp_data/Full_data/framedata.npy",allow_pickle=True)
labeldata = np.load("../sp_data/Full_data/meanlabels2.npy",allow_pickle=True)
frame_data = np.array(frame_data)
print(frame_data.shape,labeldata.shape)

(10199, 2, 256, 64) (10199,)


In [3]:
frame_data[0].sum(),labeldata[0]

(1493852, 28.105569)

In [4]:
def shuffle_data_and_labels(data, labels):
    assert len(data) == len(labels), "Data and labels must have the same number of samples"
    
    # Get the number of samples
    num_samples = len(labels)
    
    # Generate a permutation of indices
    permutation = np.random.permutation(num_samples)
    
    # Shuffle the data and labels using the permutation
    shuffled_data = data[permutation]
    shuffled_labels = labels[permutation]
    
    return shuffled_data, shuffled_labels

frame_data,labeldata = shuffle_data_and_labels(frame_data,labeldata)
frame_data[0].sum(),labeldata[0]

(1739558, 9.796058500000001)

In [5]:
print(frame_data.shape,labeldata.shape)

(10199, 2, 256, 64) (10199,)


In [6]:
frame_data_tensor = torch.tensor(frame_data,dtype=torch.float32)/255 #Create tensor and normalize
labels_tensor = torch.tensor(2*labeldata/max(labeldata)-1, dtype=torch.float32) #create tensor and normalize

In [7]:
del(frame_data) #free some memory

In [8]:
train_samples = int(0.8 * len(frame_data_tensor))
train_input = frame_data_tensor[0:train_samples]
test_input = frame_data_tensor[train_samples:]
train_output = labels_tensor[0:train_samples]
test_output = labels_tensor[train_samples:]
print(train_input.shape,train_output.shape,test_input.shape,test_output.shape)

torch.Size([8159, 2, 256, 64]) torch.Size([8159]) torch.Size([2040, 2, 256, 64]) torch.Size([2040])


In [44]:


class SpeedNet(nn.Module):
    def __init__(self, in_channels, f1,f2,f3,f4,f5):
        super(SpeedNet, self).__init__()
        
        self.batchnorm1 = nn.BatchNorm3d(f1)
        self.batchnorm2 = nn.BatchNorm3d(f2)
        self.batchnorm3 = nn.BatchNorm3d(f3)
        self.batchnorm4 = nn.BatchNorm2d(f4)
        self.batchnorm5 = nn.BatchNorm2d(f5)
        # First 3D convolution layer
        self.conv3d_1 = nn.Conv3d(in_channels, f1, kernel_size=(1, 5, 5))
        self.pool3d_1 = nn.MaxPool3d(kernel_size=(1, 1, 3), stride=(1, 1, 2))
        
        # Second 3D convolution layer
        self.conv3d_2 = nn.Conv3d(f1, f2, kernel_size=(1, 5, 5))
        self.pool3d_2 = nn.MaxPool3d(kernel_size=(1, 1, 2), stride=(1, 1, 2))
        
        # Third 3D convolution layer
        self.conv3d_3 = nn.Conv3d(f2, f3, kernel_size=(2, 5, 5))
        
        # 2D convolution layer
        self.conv2d_1 = nn.Conv2d(f3, f4, kernel_size=(10, 1))
        self.pool2d_1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(1, 2))

        self.conv2d_2 = nn.Conv2d(f4, f5, kernel_size=(1, 3))
        self.pool2d_2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(1, 2))



        self.fc1 = nn.Linear(936, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32,1)
        
    def deb_forward(self, x):
        # Reshape input tensor to add the single channel dimension
        print("Before unsqueeze:", x.shape)
        x = x.unsqueeze(1)
        print("After unsqueeze:", x.shape)
        
        # First 3D convolution and pooling
        x = self.conv3d_1(x)
        print("After conv3d_1:", x.shape)
        x = self.pool3d_1(x)
        print("After pool3d_1:", x.shape)
        
        
        # Second 3D convolution and pooling
        x = self.conv3d_2(x)
        print("After conv3d_2:", x.shape)
        x = self.pool3d_2(x)
        print("After pool3d_2:", x.shape)
        
        # Third 3D convolution
        x = self.conv3d_3(x)
        print("After conv3d_3:", x.shape)
        
        # Squeeze the third dimension
        x = x.squeeze(2)
        print("After squeeze:", x.shape)
        
        # First 2D convolution and pooling
        x = self.conv2d_1(x)
        print("After conv2d_1:", x.shape)
        x = self.pool2d_1(x)
        print("After pool2d_1:", x.shape)

        # Second 2D convolution and pooling
        x = self.conv2d_2(x)
        print("After conv2d_2:", x.shape)

        
        # Flatten the data
        x = x.view(x.size(0), -1)
        print("After flatten:", x.shape)

        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = self.fc3(x)
        
        
        return x
    
    def forward(self, x):
        # Reshape input tensor to add the single channel dimension
        #print("Before unsqueeze:", x.shape)
        x = x.unsqueeze(1)
        #print("After unsqueeze:", x.shape)
        
        # First 3D convolution and pooling
        x = self.conv3d_1(x)
        #print("After conv3d_1:", x.shape)
        x = self.pool3d_1(x)
        #print("After pool3d_1:", x.shape)
        x = self.batchnorm1(x)
        
        # Second 3D convolution and pooling
        x = self.conv3d_2(x)
        #print("After conv3d_2:", x.shape)
        x = self.pool3d_2(x)
        #print("After pool3d_2:", x.shape)
        x = self.batchnorm2(x)
        # Third 3D convolution
        x = self.conv3d_3(x)
        #print("After conv3d_3:", x.shape)
        x = self.batchnorm3(x)
        # Squeeze the third dimension
        x = x.squeeze(2)
        #print("After squeeze:", x.shape)
        
        # First 2D convolution and pooling
        x = self.conv2d_1(x)
        #print("After conv2d_1:", x.shape)
        x = self.pool2d_1(x)
        x = self.batchnorm4(x)
        #print("After pool2d_1:", x.shape)

        # Second 2D convolution and pooling
        x = self.conv2d_2(x)
        x = self.batchnorm5(x)
        #print("After conv2d_2:", x.shape)
     
        
        # Flatten the data
        x = x.view(x.size(0), -1)
        #print("After flatten:", x.shape)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        
        return x




model = SpeedNet(1, 8,16,32,64,2)  # Adjust in_channels to 1 for the first 3D convolution
input_tensor = train_input[0:2]

output = model.deb_forward(input_tensor)
output = model.forward(input_tensor)

Before unsqueeze: torch.Size([2, 2, 256, 64])
After unsqueeze: torch.Size([2, 1, 2, 256, 64])
After conv3d_1: torch.Size([2, 8, 2, 252, 60])
After pool3d_1: torch.Size([2, 8, 2, 252, 29])
After conv3d_2: torch.Size([2, 16, 2, 248, 25])
After pool3d_2: torch.Size([2, 16, 2, 248, 12])
After conv3d_3: torch.Size([2, 32, 1, 244, 8])
After squeeze: torch.Size([2, 32, 244, 8])
After conv2d_1: torch.Size([2, 64, 235, 8])
After pool2d_1: torch.Size([2, 64, 234, 4])
After conv2d_2: torch.Size([2, 2, 234, 2])
After flatten: torch.Size([2, 936])


In [37]:
output[0]

tensor([-0.0432], grad_fn=<SelectBackward0>)

In [38]:
# class Mini_SpeedNet(nn.Module):
#     def __init__(self, input_shape):
#         super(Mini_SpeedNet, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=input_shape[1], out_channels=24, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(in_channels=24, out_channels=3, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.batchnorm1 = nn.BatchNorm2d(24)
#         self.batchnorm2 = nn.BatchNorm2d(3)
#         self.flatten = nn.Flatten()
        
#         # Calculate the size of the flattened features after the convolution and pooling layers
#         conv_output_size = self._get_conv_output(input_shape)
        
#         self.fc1 = nn.Linear(conv_output_size, 64)
#         self.fc2 = nn.Linear(64, 16)
#         self.fc3 = nn.Linear(16,1)

#     def conv_forward(self, x):
#         x = self.conv1(x)
#         x = self.batchnorm1(x)
#         x = F.relu(x)
#         x = self.pool(x)
        
#         x = self.conv2(x)
#         x = self.batchnorm2(x)
#         x = F.relu(x)
#         #x = self.pool(x)
        
#         return x
    
#     def _get_conv_output(self, shape):
#         x = torch.rand(shape[1:], dtype=torch.float32).unsqueeze(0)
#         x = self.conv_forward(x)
#         sizee = int(torch.prod(torch.tensor(x.size())))
#         return sizee

#     def forward(self, x):
#         x = self.conv_forward(x)
#         x = self.flatten(x)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)

#         return x
    
# model = Mini_SpeedNet(frame_data_tensor.shape)

In [45]:
criterion = nn.MSELoss()
optimizer = optim.Adagrad(model.parameters())

In [46]:
model.train()

SpeedNet(
  (batchnorm1): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm5): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3d_1): Conv3d(1, 8, kernel_size=(1, 5, 5), stride=(1, 1, 1))
  (pool3d_1): MaxPool3d(kernel_size=(1, 1, 3), stride=(1, 1, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3d_2): Conv3d(8, 16, kernel_size=(1, 5, 5), stride=(1, 1, 1))
  (pool3d_2): MaxPool3d(kernel_size=(1, 1, 2), stride=(1, 1, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3d_3): Conv3d(16, 32, kernel_size=(2, 5, 5), stride=(1, 1, 1))
  (conv2d_1): Conv2d(32, 64, kernel_size=(10, 1), stride=(1, 1))
  (pool2d_1): MaxPool2d(kernel

In [47]:
def get_batch(tensor_data:torch.tensor,tensor_label:torch.tensor,batch_size=64):
    start_index = np.random.randint(0,len(tensor_data)-batch_size-1)
    end_index = start_index+batch_size
    return tensor_data[start_index:end_index],tensor_label[start_index:end_index] #return a view of data

def eval_batch(tensor_data:torch.tensor,tensor_label:torch.tensor,epoch,eval_batch_size = 512):
    start = np.random.randint(0,len(tensor_label)-eval_batch_size-1)
    end = start+eval_batch_size
    output_test = model(tensor_data[start:end])
    loss_test = criterion(output_test.squeeze(),tensor_label[start:end])
    print(f'Epoch {epoch+1}, Loss: {np.sqrt(loss_test.item())*max(labeldata)}')



In [48]:
epoch = 0
for epoch in range(10000):
    # Zero the parameter gradients
    optimizer.zero_grad()
    
    # Forward pass
    #random_batch = np.random.randint(5000,6000)
    x_data,y_data = get_batch(frame_data_tensor,labels_tensor,batch_size=16)
    outputs = model(x_data)
    loss = criterion(outputs.squeeze(), y_data)
    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()
    
    if epoch%100==0:
        print("Evaluating")
        eval_batch(test_input,test_output,epoch)
    if epoch%10==0:
        print(f'Epoch {epoch+1}, Loss: {np.sqrt(loss.item())*max(labeldata)}')

print('Finished Training')

Evaluating
Epoch 1, Loss: 20.667671256642496
Epoch 1, Loss: 17.454074981412823
Epoch 11, Loss: 10.499508986206925
Epoch 21, Loss: 8.040229409069923
Epoch 31, Loss: 7.8846761881943515
Epoch 41, Loss: 7.276973134988043
Epoch 51, Loss: 5.060534390613447
Epoch 61, Loss: 8.109516372771969
Epoch 71, Loss: 10.435005690167712
Epoch 81, Loss: 12.494627474111569
Epoch 91, Loss: 7.117615311196486
Evaluating
Epoch 101, Loss: 7.5466976328767945
Epoch 101, Loss: 8.30643028954413
Epoch 111, Loss: 12.116681637061419
Epoch 121, Loss: 13.904473033190818
Epoch 131, Loss: 10.142525351773017
Epoch 141, Loss: 7.456214245593057
Epoch 151, Loss: 10.709656445264375
Epoch 161, Loss: 4.882993119320764
Epoch 171, Loss: 10.338516376498639
Epoch 181, Loss: 4.794300869122748
Epoch 191, Loss: 5.775805542676842
Evaluating
Epoch 201, Loss: 6.5470925310979124
Epoch 201, Loss: 6.993432666499296
Epoch 211, Loss: 5.333276463288863
Epoch 221, Loss: 13.169918524339785
Epoch 231, Loss: 7.154495185174573
Epoch 241, Loss: 7.841

In [49]:
for epoch in range(10000,20000):
    # Zero the parameter gradients
    optimizer.zero_grad()
    
    # Forward pass
    #random_batch = np.random.randint(5000,6000)
    x_data,y_data = get_batch(frame_data_tensor,labels_tensor,batch_size=16)
    outputs = model(x_data)
    loss = criterion(outputs.squeeze(), y_data)
    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()
    
    if epoch%100==0:
        print("Evaluating")
        eval_batch(test_input,test_output,epoch)
    if epoch%10==0:
        print(f'Epoch {epoch+1}, Loss: {np.sqrt(loss.item())*max(labeldata)}')

print('Finished Training')

Evaluating
Epoch 10001, Loss: 2.914099858637356
Epoch 10001, Loss: 3.869545974511474
Epoch 10011, Loss: 2.9406415184192345
Epoch 10021, Loss: 3.012004517009309
Epoch 10031, Loss: 2.6718178857404107
Epoch 10041, Loss: 3.922567768240186
Epoch 10051, Loss: 2.758993129692507
Epoch 10061, Loss: 3.3290303261962784
Epoch 10071, Loss: 3.1228345242233746
Epoch 10081, Loss: 3.1759401330676496
Epoch 10091, Loss: 4.230361201741598
Evaluating
Epoch 10101, Loss: 2.956982179994613
Epoch 10101, Loss: 3.6579143599371413
Epoch 10111, Loss: 1.4994506638924525
Epoch 10121, Loss: 3.671458302724617
Epoch 10131, Loss: 3.2058182956473513
Epoch 10141, Loss: 3.3685366868647377
Epoch 10151, Loss: 3.1881853289610924
Epoch 10161, Loss: 2.808858130998592
Epoch 10171, Loss: 5.206559122386152
Epoch 10181, Loss: 3.1368856448432045
Epoch 10191, Loss: 2.742755248513745
Evaluating
Epoch 10201, Loss: 2.7465320393515937
Epoch 10201, Loss: 3.1620476582571126
Epoch 10211, Loss: 2.7058251187585993
Epoch 10221, Loss: 2.6697706

In [51]:
for x,y in zip(frame_data_tensor,labels_tensor):
    x = x.unsqueeze(0)
    out = model.forward(x)
    print(out,y)
    

tensor([[-0.7173]], grad_fn=<AddmmBackward0>) tensor(-0.8600)
tensor([[-0.4619]], grad_fn=<AddmmBackward0>) tensor(-0.5226)
tensor([[-0.0661]], grad_fn=<AddmmBackward0>) tensor(0.0893)
tensor([[-0.4872]], grad_fn=<AddmmBackward0>) tensor(-0.4520)
tensor([[-0.0105]], grad_fn=<AddmmBackward0>) tensor(0.0200)
tensor([[-0.7481]], grad_fn=<AddmmBackward0>) tensor(-0.7905)
tensor([[-0.9477]], grad_fn=<AddmmBackward0>) tensor(-0.8508)
tensor([[-0.0238]], grad_fn=<AddmmBackward0>) tensor(-0.2766)
tensor([[-0.5705]], grad_fn=<AddmmBackward0>) tensor(-0.7100)
tensor([[-0.5310]], grad_fn=<AddmmBackward0>) tensor(-0.9158)
tensor([[-0.7649]], grad_fn=<AddmmBackward0>) tensor(-1.)
tensor([[-0.6215]], grad_fn=<AddmmBackward0>) tensor(-0.6346)
tensor([[-0.6998]], grad_fn=<AddmmBackward0>) tensor(-0.7321)
tensor([[0.8169]], grad_fn=<AddmmBackward0>) tensor(0.5803)
tensor([[-0.1530]], grad_fn=<AddmmBackward0>) tensor(-0.5388)
tensor([[-0.2513]], grad_fn=<AddmmBackward0>) tensor(-0.2215)
tensor([[-0.2241

KeyboardInterrupt: 

In [50]:
torch.save(model.state_dict(), '../sp_data/Full_data/SpeedNet_64.pth') #Has 64bins


### Previously the speed bins were 64 hence the Speed res was less
### Now increase the features for 10 x kernel to 128

In [31]:
import torch
import torch.nn as nn

class SpeedNet2(nn.Module):
    def __init__(self, in_channels, f1,f2,f3,f4,f5):
        super(SpeedNet2, self).__init__()
        
        self.batchnorm1 = nn.BatchNorm3d(f1)
        self.batchnorm2 = nn.BatchNorm3d(f2)
        self.batchnorm3 = nn.BatchNorm3d(f3)
        self.batchnorm4 = nn.BatchNorm2d(f4)
        self.batchnorm5 = nn.BatchNorm2d(f5)
        # First 3D convolution layer
        self.conv3d_1 = nn.Conv3d(in_channels, f1, kernel_size=(1, 5, 5))
        self.pool3d_1 = nn.MaxPool3d(kernel_size=(1, 1, 3), stride=(1, 1, 2))
        
        # Second 3D convolution layer
        self.conv3d_2 = nn.Conv3d(f1, f2, kernel_size=(1, 5, 5))
        self.pool3d_2 = nn.MaxPool3d(kernel_size=(1, 1, 2), stride=(1, 1, 2))
        
        # Third 3D convolution layer
        self.conv3d_3 = nn.Conv3d(f2, f3, kernel_size=(2, 5, 5))
        
        # 2D convolution layer
        self.conv2d_1 = nn.Conv2d(f3, f4, kernel_size=(10, 7))
        self.pool2d_1 = nn.MaxPool2d(kernel_size=(9, 1), stride=(9, 2))

        self.conv2d_2 = nn.Conv2d(f4, f5, kernel_size=(5, 1))
        self.pool2d_2 = nn.MaxPool2d(kernel_size=(5, 1), stride=(5, 1))

        



        self.fc1 = nn.Linear(1024, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32,1)
        
    def deb_forward(self, x):
        # Reshape input tensor to add the single channel dimension
        print("Before unsqueeze:", x.shape)
        x = x.unsqueeze(1)
        print("After unsqueeze:", x.shape)
        
        # First 3D convolution and pooling
        x = self.conv3d_1(x)
        print("After conv3d_1:", x.shape)
        x = self.pool3d_1(x)
        print("After pool3d_1:", x.shape)
        
        
        # Second 3D convolution and pooling
        x = self.conv3d_2(x)
        print("After conv3d_2:", x.shape)
        x = self.pool3d_2(x)
        print("After pool3d_2:", x.shape)
        
        # Third 3D convolution
        x = self.conv3d_3(x)
        print("After conv3d_3:", x.shape)
        
        # Squeeze the third dimension
        x = x.squeeze(2)
        print("After squeeze:", x.shape)
        
        # First 2D convolution and pooling
        x = self.conv2d_1(x)
        print("After conv2d_1:", x.shape)
        x = self.pool2d_1(x)
        print("After pool2d_1:", x.shape)

        # Second 2D convolution and pooling
        x = self.conv2d_2(x)
        print("After conv2d_2:", x.shape)

        x = self.pool2d_2(x)
        print("After pool2d_2:", x.shape)

        
        # Flatten the data
        x = x.view(x.size(0), -1)
        print("After flatten:", x.shape)

        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))fter conv2d_1: torch.Size([2, 128, 235, 2
        # x = self.fc3(x)
        
        
        return x
    
    def forward(self, x):
        # Reshape input tensor to add the single channel dimension
        #print("Before unsqueeze:", x.shape)
        x = x.unsqueeze(1)
        #print("After unsqueeze:", x.shape)
        
        # First 3D convolution and pooling
        x = self.conv3d_1(x)
        #print("After conv3d_1:", x.shape)
        x = self.pool3d_1(x)
        #print("After pool3d_1:", x.shape)
        x = self.batchnorm1(x)
        
        # Second 3D convolution and pooling
        x = self.conv3d_2(x)
        #print("After conv3d_2:", x.shape)
        x = self.pool3d_2(x)
        #print("After pool3d_2:", x.shape)
        x = self.batchnorm2(x)
        # Third 3D convolution
        x = self.conv3d_3(x)
        #print("After conv3d_3:", x.shape)
        x = self.batchnorm3(x)
        # Squeeze the third dimension
        x = x.squeeze(2)
        #print("After squeeze:", x.shape)
        
        # First 2D convolution and pooling
        x = self.conv2d_1(x)
        #print("After conv2d_1:", x.shape)
        x = self.pool2d_1(x)
        x = self.batchnorm4(x)
        #print("After pool2d_1:", x.shape)

        # Second 2D convolution and pooling
        x = self.conv2d_2(x)
        x = self.pool2d_2(x)
        x = self.batchnorm5(x)
        #print("After conv2d_2:", x.shape)
        
        
        # Flatten the data
        x = x.view(x.size(0), -1)
        #print("After flatten:", x.shape)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        
        return x




model = SpeedNet2(1, 8,16,32,128,256)  # Increased features to 128 
input_tensor = train_input[0:2]

output = model.deb_forward(input_tensor)
output = model.forward(input_tensor)

Before unsqueeze: torch.Size([2, 2, 256, 64])
After unsqueeze: torch.Size([2, 1, 2, 256, 64])
After conv3d_1: torch.Size([2, 8, 2, 252, 60])
After pool3d_1: torch.Size([2, 8, 2, 252, 29])
After conv3d_2: torch.Size([2, 16, 2, 248, 25])
After pool3d_2: torch.Size([2, 16, 2, 248, 12])
After conv3d_3: torch.Size([2, 32, 1, 244, 8])
After squeeze: torch.Size([2, 32, 244, 8])
After conv2d_1: torch.Size([2, 128, 235, 2])
After pool2d_1: torch.Size([2, 128, 26, 1])
After conv2d_2: torch.Size([2, 256, 22, 1])
After pool2d_2: torch.Size([2, 256, 4, 1])
After flatten: torch.Size([2, 1024])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x1024 and 768x64)

In [27]:
criterion = nn.MSELoss()
optimizer = optim.Adagrad(model.parameters())
model.train()

/home/akshayd/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


SpeedNet2(
  (batchnorm1): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3d_1): Conv3d(1, 8, kernel_size=(1, 5, 5), stride=(1, 1, 1))
  (pool3d_1): MaxPool3d(kernel_size=(1, 1, 3), stride=(1, 1, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3d_2): Conv3d(8, 16, kernel_size=(1, 5, 5), stride=(1, 1, 1))
  (pool3d_2): MaxPool3d(kernel_size=(1, 1, 2), stride=(1, 1, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3d_3): Conv3d(16, 32, kernel_size=(2, 5, 5), stride=(1, 1, 1))
  (conv2d_1): Conv2d(32, 128, kernel_size=(10, 7), stride=(1, 1))
  (pool2d_1): MaxPool2d(k

In [28]:
def get_batch(tensor_data:torch.tensor,tensor_label:torch.tensor,batch_size=64):
    start_index = np.random.randint(0,len(tensor_data)-batch_size-1)
    end_index = start_index+batch_size
    return tensor_data[start_index:end_index],tensor_label[start_index:end_index] #return a view of data

def eval_batch(tensor_data:torch.tensor,tensor_label:torch.tensor,epoch,eval_batch_size = 256):
    start = np.random.randint(0,len(tensor_label)-eval_batch_size-1)
    end = start+eval_batch_size
    output_test = model(tensor_data[start:end])
    loss_test = criterion(output_test.squeeze(),tensor_label[start:end])
    print(f'Epoch {epoch+1}, Loss: {np.sqrt(loss_test.item())*max(labeldata)}')



In [29]:
epoch = 0
for epoch in range(30000):
    # Zero the parameter gradients
    optimizer.zero_grad()
    
    # Forward pass
    #random_batch = np.random.randint(5000,6000)
    x_data,y_data = get_batch(frame_data_tensor,labels_tensor,batch_size=16)
    outputs = model(x_data)
    loss = criterion(outputs.squeeze(), y_data)
    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()
    
    if epoch%100==0:
        print("Evaluating")
        eval_batch(test_input,test_output,epoch)
    if epoch%20==0:
        print(f'Epoch {epoch+1}, Loss: {np.sqrt(loss.item())*max(labeldata)}')

print('Finished Training')

Evaluating
Epoch 1, Loss: 16.589617124678814
Epoch 1, Loss: 19.682605804917
Epoch 21, Loss: 10.282722945718856
Epoch 41, Loss: 11.226940825752548


KeyboardInterrupt: 